In [3]:
embeding_matrix = load_pickle_file("../../new_disk/embeding_matrix")
word2index = load_pickle_file("../../new_disk/word2index")
index_sequences = load_pickle_file("../../new_disk/index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 434
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3000440

In [4]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)
labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [30]:
labels

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

Dataset split

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [6]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>LSTM</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [8]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [17]:
LSTM_HIDDEN_DIM_SIZE = 64
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
lstm_model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
lstm_model.add(embedding_layer)
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
lstm_model.add(Dense(LSTM_HIDDEN_DIM_SIZE//4, activation = 'relu'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//4, activation = 'tanh',dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
lstm_model.add(Dense(2,activation='sigmoid'))

In [40]:
lstm_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 434, 300)          900132300 
_________________________________________________________________
unified_lstm_14 (UnifiedLSTM (None, 434, 64)           93440     
_________________________________________________________________
unified_lstm_15 (UnifiedLSTM (None, 32)                12416     
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 34        
Total params: 900,238,718
Trainable params: 106,418
Non-trainable params: 900,132,300
_________________________________________________________________


In [16]:
lstm_model.reset_states()

In [18]:
opt = tk.optimizers.Adam(learning_rate=0.01)

In [19]:
lstm_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
lstm_model.fit(data_train,labels_train,epochs = 25,batch_size = 512, validation_data = [data_val,labels_val], callbacks = [model_checkpoint])

Train on 7380 samples, validate on 821 samples
Epoch 1/25
7380/7380 [==============================] - 60s 8ms/sample - loss: 0.6888 - acc: 0.5354 - val_loss: 0.6828 - val_acc: 0.5152
Epoch 2/25
7380/7380 [==============================] - 51s 7ms/sample - loss: 0.6822 - acc: 0.5449 - val_loss: 0.7402 - val_acc: 0.5189
Epoch 3/25
7380/7380 [==============================] - 48s 7ms/sample - loss: 0.6813 - acc: 0.5589 - val_loss: 0.6840 - val_acc: 0.4836
Epoch 4/25
7380/7380 [==============================] - 86s 12ms/sample - loss: 0.6764 - acc: 0.5701 - val_loss: 0.6725 - val_acc: 0.5518
Epoch 5/25
7380/7380 [==============================] - 49s 7ms/sample - loss: 0.6909 - acc: 0.5178 - val_loss: 0.6867 - val_acc: 0.5268
Epoch 6/25
7380/7380 [==============================] - 49s 7ms/sample - loss: 0.6892 - acc: 0.5262 - val_loss: 0.6875 - val_acc: 0.5140
Epoch 7/25
7380/7380 [==============================] - 52s 7ms/sample - loss: 0.6881 - acc: 0.5376 - val_loss: 0.6864 - val_acc: 

In [20]:
lstm_model.save("./Checkpoints/lstm_model.h5")

In [21]:
lstm_model.save_weights("./Checkpoints/lstm_weights")

In [23]:
from sklearn.metrics import accuracy_score

pred = lstm_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [39]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],p,pos_label=0)

0.9874812218327491

In [34]:
accuracy_score([np.argmax(x) for x in y_test],p)

0.9878108239882983

[1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,


In [19]:
i=0
for seq in data_train[128:250]:
    for el in seq:
        if np.isnan(el):
            print(i)
                
#         if type(arr) is np.ndarray:
#             print(i)
    i+=1

In [2]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
